<a href="https://colab.research.google.com/github/FlarPet/github-project-demo/blob/master/Copy_of_pca_tsne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dimensionality Reduction

This is the hands on part of the lecture. The hands on is separated into two sections.

# Section 1

Use Principal Components Analysis (PCA) on RNA-seq data

In [ ]:
# import packages
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import urllib.request

import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA

### Download Breast Cancer Gene Expression

This is a dataset of 1100 RNA-seq samples from TCGA. The samples can be devided into 4 distinct subtypes (Luminal A, Luminal B, Her2+, and Triple negative)

In [ ]:
urllib.request.urlretrieve("https://mssm-share.s3.amazonaws.com/data_mrna_seq_v2_rsem.txt", "rna_seq_breast_cancer.txt")
rna_seq = pd.read_csv("rna_seq_breast_cancer.txt", sep="\t").set_index("Hugo_Symbol").iloc[:,1:]
rna_seq.index = rna_seq.index.astype(str)

### Calculate PCA and plot the top two components

Running on the RNA-seq data we compute the top two PCs. These PCs carry the most variance in the orioginal data.

In [ ]:
pca = PCA(n_components=20)

pt = pca.fit_transform(rna_seq.T)

plt.scatter(pt[:,0], pt[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")

### Plot the variance exmplained by the PCs

The PCs are sorted based on the variance that they capture. PCs which explain a large fraction of variance are often considered more important than PCs that only capture a fraction of the variance.

In [ ]:
pca.explained_variance_ratio_[0:20]

In [ ]:
labels = ["PC"+str(x) for x in list(range(1,21)[::-1])]
p = plt.barh(list(range(20)), width=pca.explained_variance_ratio_[::-1])
p = plt.yticks(list(range(20)), labels=labels)
t = plt.xlabel("variance explained (%)", fontsize=16)

# Section 2

Use t-SNE and UMAP and compare to PCA

### Run t-SNE on RNA-seq data

Running t-SNE results in a different representation of the orginal data and the PCA projection. The perplexity defines the number of neighbors to be considered.

The resulting plot is more pleasing to look at, but it should be noted that neither is better or worse.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=20)
transformed_data = tsne.fit_transform(rna_seq.T)

p = plt.scatter(transformed_data[:,0], transformed_data[:,1])

## Download MNIST data

The MNIST data is a database of handwritten digits from 0 to 9. It is a popular dataset for machine learning purposes.

In [ ]:
urllib.request.urlretrieve("https://mssm-share.s3.amazonaws.com/mnist_test.csv", "mnist.csv")

mnist = np.array(pd.read_csv("mnist.csv"))
X = mnist[:, 1:]
y = mnist[:, 0]

Define some parameters for the machine learning algorithm we want to use.

In [ ]:
# Set the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',  # the type of boosting algorithm to use
    'objective': 'multiclass',  # multiclass classification
    'metric': 'multi_logloss',  # evaluation metric
    'num_class': 10,  # number of classes
    'num_leaves': 31,  # number of leaves in each tree
    'learning_rate': 0.05,  # the learning rate
    'feature_fraction': 0.9,  # fraction of features to use for each iteration
    'bagging_fraction': 0.8,  # fraction of data to use for each iteration
    'bagging_freq': 5,  # frequency of bagging
    'verbose': -1  # logging mode - quiet
}

num_round = 100

Now we train a machine learning model and predict the labels of some training data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Convert the data into LightGBM's data format
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the model
bst = lgb.train(params, train_data, num_boost_round=num_round, valid_sets=[test_data], verbose_eval = -1)

# Predict using the trained model
y_pred = bst.predict(X_test)

# Convert the predicted probabilities into class labels
y_pred_classes = y_pred.argmax(axis=1)

accuracy = np.mean(y_pred_classes == y_test)
print("Accuracy:", accuracy)

lightGBM was able to predict the correct digit label in 94.6% of cases. When trying to build any kind new kind of sophisticated new model always start with a simple out of the box solution as a reference. As we can see the prediction works already really well just using the full 784 dimensions.

In [ ]:
tsne = TSNE(n_components=3, perplexity=20)
transformed_data = tsne.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformed_data, y, test_size=0.2, random_state=1)

# Convert the data into LightGBM's data format
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the model
bst = lgb.train(params, train_data, num_boost_round=num_round, valid_sets=[test_data], verbose_eval=-1)

# Predict using the trained model
y_pred = bst.predict(X_test)

# Convert the predicted probabilities into class labels
y_pred_classes = y_pred.argmax(axis=1)

accuracy = np.mean(y_pred_classes == y_test)
print("Accuracy t-SNE:", accuracy)

Computing the 3D t-SNE was quite slow. But after transforming the data to only 3 dimensions the prediction accuracy using the same machine learnin algorithm was improved to 95.1%. 

Next we install the UMAP package. UMAP ios very similar to t-SNE, but it is probably the better method to use nowadays. It is much faster and can handle higher dimensions. It also is better at keeping the relationships between datapoints more similar compared to t-SNE.

In [ ]:
!pip install umap-learn --quiet

After installing UMAP we can test the prediction performance based on UMAP transformed data. The syntax should be identical to t-SNE.

In [ ]:
import umap
import matplotlib.patches as mpatches

# Perform UMAP
umap_model = umap.UMAP(n_components=2)
X_umap = umap_model.fit_transform(X)

plt.scatter(X_umap[:,0], X_umap[:,1], c=y, cmap='viridis')

patches = [mpatches.Patch(color=plt.cm.viridis(i/10), 
            label=str(i)) for i in range(10)]

plt.legend(handles=patches, 
            bbox_to_anchor=(1.05, 1), 
            loc=2, 
            borderaxespad=0.0)

plt.xlabel("T1", fontsize=16)
plt.ylabel("T2", fontsize=16)

In [ ]:
import umap

# Perform UMAP
umap_model = umap.UMAP(n_components=20)
X_umap = umap_model.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_umap, y, test_size=0.2, random_state=1)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the model
bst = lgb.train(params, train_data, num_boost_round=num_round, valid_sets=[test_data], verbose_eval=-1)

# Predict using the trained model
y_pred = bst.predict(X_test)

# Convert the predicted probabilities into class labels
y_pred_classes = y_pred.argmax(axis=1)

accuracy = np.mean(y_pred_classes == y_test)
print("Accuracy UMAP:", accuracy)

This transformation seems to improve the prediction accuracy slightly. The higher dimensions might help. It should be noted that we do not have proof that the resulting accuracy is statistically significant.

When inspecting the mislabeled digits most can be identified by a human. Here is also a list of best performers: https://paperswithcode.com/sota/image-classification-on-mnist

The human error rate is 2%-2.5%:
https://papers.nips.cc/paper/656-efficient-pattern-recognition-using-a-new-transformation-distance.pdf


In [ ]:
l = 15  # change integer to show different mislabeled samples

false_predictions = np.where(y_pred_classes != y_test)[0]
i = np.where(X_umap[:,0] == X_test[false_predictions[l],0])[0]
print("Thought it was:",y_pred_classes[false_predictions[l]], ", but it was:", y_test[false_predictions[l]])

plt.imshow(1-X[i, :].reshape(28, 28), cmap='gray')
t = plt.xticks([])
t = plt.yticks([])

There is a risk that we remove important information when using methods such as UMAP. It should be noted that most modern machine learning algorithms can handle with many dimensions very well. Instead of reducing the number of dimensions we add the UMAP dimensions to the original data and see whether this can improve out prediction even more.

In [ ]:
umap_model = umap.UMAP(n_components=2)
X_umap = umap_model.fit_transform(X)

X_augmented = np.concatenate((X, X_umap), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_augmented, y, test_size=0.2, random_state=1)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the model
bst = lgb.train(params, train_data, num_boost_round=num_round, valid_sets=[test_data], verbose_eval=-1)

# Predict using the trained model
y_pred = bst.predict(X_test)

# Convert the predicted probabilities into class labels
y_pred_classes = y_pred.argmax(axis=1)

accuracy = np.mean(y_pred_classes == y_test)
print("Accuracy + UMAP:", accuracy)

### Differences between PCA, t-SNE, and UMAP

All three algorithms perform slightly different due to the underlying principles they are based on. In the following section we analyze a synthetic dataset to illustrate the differences.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import urllib.request

import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA

In [ ]:
urllib.request.urlretrieve("https://mssm-share.s3.amazonaws.com/hands_on_data.tsv", "hands_on_data.tsv")
syn_data = pd.read_csv("hands_on_data.tsv", index_col=0, sep="\t")

## PCA

The first two PCs describe a circle and it is hard to make out a relationship with the color gradient and the structure of the data.

In [ ]:
colors = syn_data.iloc[:,3]

pca = PCA(n_components=2)
pca_data = pca.fit_transform(syn_data.iloc[:,0:3])

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(pca_data[:,0], pca_data[:,1], c=colors, cmap='hsv', marker='o')
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_title('PCA')

## t-SNE



In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=20)
tsne_data = tsne.fit_transform(syn_data.iloc[:,0:3])

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(tsne_data[:,0], tsne_data[:,1], c=syn_data.iloc[:,3], cmap='hsv', marker='o')
ax.set_xlabel('T1')
ax.set_ylabel('T2')
ax.set_title('TSNE')

## UMAP

In [ ]:
import umap

umap_model = umap.UMAP()
umap_data = umap_model.fit_transform(syn_data.iloc[:,0:3])

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(umap_data[:,0], umap_data[:,1], c=syn_data.iloc[:,3], cmap='hsv', marker='o')
ax.set_xlabel('U1')
ax.set_ylabel('U2')
ax.set_title('UMAP')

In [ ]:
!pip install bokeh --quiet
!pip install datashader --quiet

In [ ]:
import umap.plot

umap.plot.connectivity(umap_model)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(syn_data['x'], syn_data['y'], syn_data['z'], c=syn_data['c'], cmap='hsv', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')